In [ ]:
# coding:utf-8
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

def gragh(result, epoch):    
    x = np.arange(0, epoch, 1)
    left = np.array(x)
    height = np.array(result)
    plt.plot(left, height)

In [ ]:
from deep_qnetwork_with_cnn import DeepQNetworkWithCNN
import random
import time
import copy
import numpy as np

ACTION = [0, 1, 2, 3, 4, 5]
NUM_HID1 = 288
NUM_HID2 = 100
NUM_OUT  = 6
BATCH_SIZE = 32
EPOCH = 12000
GAMMA = 0.9
EPSIL = 1.0
MINEPSIL = 0.1
NUM_LIMIT_RECORD = 2000
NUM_INIT_RECORD = 40000
ProccerUnit = "CPU"
MODEL_NAME = "./DQN_model/breakout1.model"
RESULT_REWARD = []

STATE_LENGTH = 4  # 状態を構成するフレーム数
FRAME_WIDTH = 84  # リサイズ後のフレーム幅
FRAME_HEIGHT = 84  # リサイズ後のフレーム高さ
NO_OP_STEPS = 30

def agent_action(env, action):
    next_state, reward, terminal, info = env.step(action)
    return next_state, reward, terminal, info

def deep_q_learn(env, agent):
    agent.load_weight()
    agent.update_target_network()
    step = 0
    for episode in range(EPOCH):
        sum_reward = 0
        terminal = False  # エピソード終了判定を初期化
        next_obs = env.reset()  # 環境の初期化、初期画面を返す
        for _ in range(random.randint(1, NO_OP_STEPS)):  # ランダムなフレーム数分「何もしない」行動で遷移させる
            now_obs = next_obs
            next_obs, _, _, _ = env.step(0)  # 「何もしない」行動を取り、次の画面を返す
        now_state = agent.get_initial_state(now_obs, next_obs)  # 初期状態を作る
        while(not terminal):
            # 状態更新
            now_obs = copy.deepcopy(next_obs)
            # 方策に従って行動を選択
            action, _, action_list = agent.policy_egreedy(now_state, agent.Q_network, step)
            # 行動による次状態を観測
            next_obs, reward, term, _ = agent_action(env, ACTION[action])
            # ゴールの設定
            terminal, reward = (True, reward) if term else (False, reward)
            
            preprocess_obs = agent.pre_proccesing(now_obs, next_obs)
            next_state = np.append(now_state[1:, :, :], np.array([preprocess_obs]), axis=0)
            sum_reward += reward
            agent.stock_record(now_state, action, next_state, reward, terminal, action_list)

            env.render()
            if terminal:
                break
                
            # 入力状態更新
            now_state = copy.deepcopy(next_state)

            # Experience_Replayによるバッチ学習
            if len(agent.record) > NUM_LIMIT_RECORD:
                agent.init_record()
                if step % agent.action_interval == 0:
                    agent.experience_replay()
                    if agent.EPSIL > MINEPSIL: agent.EPSIL -= EPSIL / EPOCH
            if step % 1000 == 0:
                agent.update_target_network()               
            step += 1

        print(episode, sum_reward, agent.EPSIL, step)
        RESULT_REWARD.append(sum_reward)
        agent.save_weight()
    gragh(RESULT_REWARD, len(RESULT_REWARD))

if __name__=="__main__":
    import gym
    env = gym.make("Breakout-v0")
    agent = DeepQNetworkWithCNN(NUM_HID1, NUM_HID2, NUM_OUT,ProccerUnit, EPSIL, GAMMA, BATCH_SIZE, MODEL_NAME)
    deep_q_learn(env, agent)

[2017-06-06 01:53:19,232] Making new env: Breakout-v0


0 3.0 1.0 311
1 2.0 1.0 611
2 0.0 1.0 825
3 2.0 1.0 1125
4 0.0 1.0 1309
5 0.0 1.0 1506
6 2.0 1.0 1801


In [ ]:
# coding-utf-8
from deep_qnetwork import DeepQNetwork
import random
import copy

NUM_HID1 = 100
NUM_HID2 = 100
NUM_HID3 = 100
NUM_OUT  = 6
BATCH_SIZE = 200
EPOCH = 1000
GAMMA = 0.9
EPSIL = 1.0
NUM_LIMIT_RECORD = 1000
NUM_INIT_RECORD = 10000
ProccerUnit = "CPU"
MODEL_NAME = "./DQN_model/"
ACTION = []
RESULT_REWARD = []

def agent_action(env, action):
    next_state, reward, terminal, info = env.step(action)
    return next_state, reward, terminal, info

def deep_Q_learn(env, Agent):
    STEP = 200
    Agent.load_weight()
    for episode in range(1, EPOCH+1):
        now_state = env.reset()
        sum_reward = 0
        for step in range(1, STEP+1):
            # 方策に従って行動を選択
            action, _, action_list = Agent.policy_egreedy(now_state, Agent.Q_network)
            # 行動による次状態を観測
            next_state, reward, term, _ = agent_action(env, [ACTION[action]])
            # ゴールの設定
            terminal, reward = (True, 1.0) if term else (False, reward)
    
            sum_reward += reward
            Agent.stock_record(now_state, action, next_state, reward, terminal, action_list)
                
            # 状態更新
            now_state = copy.deepcopy(next_state)
            # env.render()
            
            if terminal:
                break
            
            # Experience_Replayによるバッチ学習
            if len(Agent.record) > NUM_LIMIT_RECORD:
                Agent.init_record()
                if step % 20 == 0:
                    Agent.experience_replay_ver2()
                    Agent.EPSIL -= EPSIL / EPOCH
                if step % 40 == 0:
                    Agent.update_target_network()               
                    Agent.save_weight()
                
        print(episode, sum_reward, Agent.EPSIL)
        RESULT_REWARD.append(sum_reward)
    gragh(RESULT_REWARD, len(RESULT_REWARD))
                
def run():
    import gym
    env = gym.make("Pendulum-v0")
    Agent = DeepQNetwork(NUM_IN, NUM_HID1, NUM_HID2, NUM_HID3, NUM_OUT,
                         ProccerUnit, EPSIL, GAMMA, BATCH_SIZE, MODEL_NAME)
    deep_Q_learn(env, Agent)